In [25]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json


from api_keys import g_key

#  Store Part I results into DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bluff,-46.6000,168.3333,57.16,66,99,10.45,NZ,1643352719
1,Mataura,-46.1927,168.8643,63.77,48,100,3.47,NZ,1643352719
2,Arman',59.7000,150.1667,5.22,78,100,16.26,RU,1643352719
3,Yanam,16.7333,82.2167,84.60,58,62,7.85,IN,1643352720
4,Hilo,19.7297,-155.0900,71.33,85,20,4.61,US,1643352721
...,...,...,...,...,...,...,...,...,...
553,Severo-Kuril'sk,50.6789,156.1250,24.30,71,100,35.86,RU,1643352770
554,Chuchar-kana Mandi,31.7500,73.8000,60.91,28,4,5.10,PK,1643353006
555,Kibala,9.1104,18.3463,64.35,45,0,2.30,TD,1643353006
556,Seoul,37.5683,126.9778,38.59,31,0,10.36,KR,1643352750


#  Humidity Heatmap

In [3]:
gmaps.configure(api_key = g_key)

locations = weather_data[["Lat","Lng"]]

humidity = weather_data["Humidity"]

In [4]:
fig = gmaps.figure(center=(46,-5), zoom_level=2)
max_intensity = np.max(humidity)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

#  Create new DataFrame fitting weather criteria

In [7]:
narrow_cities=weather_data.loc[(weather_data["Wind Speed"]<= 10) & (weather_data["Cloudiness"] == 0) & \
                               (weather_data["Max Temp"] >=70) & (weather_data["Max Temp"] <=80)].dropna()

narrow_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,Kapaa,22.0752,-159.3190,71.58,70,0,1.01,US,1643352505
294,Sur,22.5667,59.5289,75.22,36,0,4.07,OM,1643352863
379,La Romana,18.4273,-68.9728,71.33,91,0,7.02,DO,1643352898
469,Porto Velho,-8.7619,-63.9039,75.25,94,0,0.00,BR,1643352859
512,Verāval,20.9000,70.3667,76.89,30,0,7.54,IN,1643352985
537,Devarkonda,16.7000,78.9333,77.07,25,0,9.15,IN,1643353000


#  Hotel Map


In [17]:
hotel_df = narrow_cities.loc[:,["City", "Country", "Lat", "Lng"]]



hotel_df["Hotel Name"] = ""

hotel_df


,City,Country,Lat,Lng,Hotel Name
27,Kapaa,US,22.0752,-159.3190,
294,Sur,OM,22.5667,59.5289,
379,La Romana,DO,18.4273,-68.9728,
469,Porto Velho,BR,-8.7619,-63.9039,
512,Verāval,IN,20.9000,70.3667,
537,Devarkonda,IN,16.7000,78.9333,


In [22]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [26]:
for index, row in hotel_df.iterrows():
    lat= row["Lat"]
    lng= row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Gathering Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    time.sleep(1)
    
print("----End of Search----")


Gathering Results for Index 27: Kapaa.
Closest hotel in Kapaa is Kauai Shores Hotel.
------------
Gathering Results for Index 294: Sur.
Closest hotel in Sur is Sur Plaza Hotel.
------------
Gathering Results for Index 379: La Romana.
Closest hotel in La Romana is Olimpo Hotel.
------------
Gathering Results for Index 469: Porto Velho.
Closest hotel in Porto Velho is Hotel Slaviero Essential Porto Velho.
------------
Gathering Results for Index 512: Verāval.
Closest hotel in Verāval is Lords Inn Somnath.
------------
Gathering Results for Index 537: Devarkonda.
Closest hotel in Devarkonda is PRR Lodge.
------------
----End of Search----


In [27]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
27,Kapaa,US,22.0752,-159.3190,Kauai Shores Hotel
294,Sur,OM,22.5667,59.5289,Sur Plaza Hotel
379,La Romana,DO,18.4273,-68.9728,Olimpo Hotel
469,Porto Velho,BR,-8.7619,-63.9039,Hotel Slaviero Essential Porto Velho
512,Verāval,IN,20.9000,70.3667,Lords Inn Somnath
537,Devarkonda,IN,16.7000,78.9333,PRR Lodge


In [28]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))